In [3]:
import cv2
import numpy as np
from skimage.morphology import skeletonize
from collections import deque
import math

def preprocess_mask(img):
    # img: BGR image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # adapt as needed: blur and then threshold
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, th = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # If cable is darker than background, invert:
    if np.mean(gray[th==255]) < np.mean(gray[th==0]):
        th = cv2.bitwise_not(th)
    # morphological cleanups
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel, iterations=2)
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations=1)
    return th

def mask_to_skeleton(mask):
    # skeletonize expects boolean image
    bw = (mask > 0)
    skel = skeletonize(bw)  # boolean array
    skel = skel.astype(np.uint8) * 255
    return skel

def find_endpoints(skel):
    # endpoints: skeleton pixels with only 1 neighbor
    h, w = skel.shape
    ends = []
    arr = skel // 255
    for y in range(1, h-1):
        for x in range(1, w-1):
            if arr[y,x]:
                neigh = arr[y-1:y+2, x-1:x+2]
                if np.sum(neigh) - 1 == 1:  # only one neighbor aside from itself
                    ends.append((x,y))
    return ends

def walk_skeleton(skel):
    # find a path along the skeleton from one endpoint to another (longest if branches)
    arr = (skel//255).astype(np.uint8)
    endpoints = find_endpoints(skel)
    if len(endpoints) == 0:
        # can be a loop (no endpoints). pick any pixel and do graph traversal to get a cycle length approx.
        ys, xs = np.where(arr)
        if len(xs)==0:
            return [], 0.0
        start = (xs[0], ys[0])
    else:
        # pick an endpoint as start
        start = endpoints[0]

    # BFS/DFS to extract a path: we'll do simple neighbor-walking preferring unvisited pixels
    visited = set()
    path = []
    x0,y0 = start
    stack = [(x0,y0)]
    while stack:
        x,y = stack.pop()
        if (x,y) in visited: 
            continue
        visited.add((x,y))
        path.append((x,y))
        # push neighbors
        for dy in (-1,0,1):
            for dx in (-1,0,1):
                if dx==0 and dy==0: continue
                nx, ny = x+dx, y+dy
                if nx<0 or ny<0 or ny>=arr.shape[0] or nx>=arr.shape[1]: continue
                if arr[ny, nx] and (nx,ny) not in visited:
                    stack.append((nx,ny))
    # path now contains visited pixels in visitation order (not strictly ordered along centerline but ok for length sum)
    # For better ordering, if endpoints exist, we could walk from start along single neighbor choices.
    return path

def ordered_path_from_endpoints(skel):
    arr = (skel//255).astype(np.uint8)
    ends = find_endpoints(skel)
    if len(ends) >= 2:
        start = ends[0]
        # greedy walk along neighbors preferring unvisited
        visited = set()
        path = [start]
        current = start
        visited.add(current)
        while True:
            x,y = current
            neighbors = []
            for dy in (-1,0,1):
                for dx in (-1,0,1):
                    if dx==0 and dy==0: continue
                    nx, ny = x+dx, y+dy
                    if nx<0 or ny<0 or ny>=arr.shape[0] or nx>=arr.shape[1]: continue
                    if arr[ny,nx] and (nx,ny) not in visited:
                        neighbors.append((nx,ny))
            if not neighbors:
                break
            # choose neighbor with smallest degree first (prefer straight)
            neighbors.sort(key=lambda p: sum(arr[p[1]+dy, p[0]+dx] 
                                           for dx in (-1,0,1) for dy in (-1,0,1)) if 0<p[0]<arr.shape[1]-1 and 0<p[1]<arr.shape[0]-1 else 0)
            nextp = neighbors[0]
            path.append(nextp)
            visited.add(nextp)
            current = nextp
        return path
    else:
        return walk_skeleton(skel)

def path_length_pixels(path):
    if len(path) < 2:
        return 0.0
    length = 0.0
    for i in range(1, len(path)):
        x1,y1 = path[i-1]
        x2,y2 = path[i]
        length += math.hypot(x2-x1, y2-y1)
    return length

if __name__ == "__main__":
    img = cv2.imread(r"c:\projects\ai_quality_inspection\notebook\spiral_img.jpg")  # provide your image
    mask = preprocess_mask(img)
    skel = mask_to_skeleton(mask)
    path = ordered_path_from_endpoints(skel)
    pix_len = path_length_pixels(path)
    print("Length in pixels:", pix_len)

    # Example: if you placed a ruler of 30 cm whose measured pixel length is ref_pixels:
    ref_pixels = 300  # measure pixel length of reference in your image
    ref_meters = 0.30
    length_m = pix_len * (ref_meters / ref_pixels)
    print("Estimated length (meters):", length_m)

    # optional: visualize skeleton and path
    vis = img.copy()
    for x,y in path:
        cv2.circle(vis, (x,y), 1, (0,0,255), -1)
    cv2.imwrite("cable_path_overlay.jpg", vis)


Length in pixels: 4248.593288050014
Estimated length (meters): 4.248593288050015
